# Map Click Polygon Intersection Example

This notebook demonstrates how to:
1. Observe map clicks using the `clicked` attribute
2. Find intersecting polygons from a GeoJSON file
3. Load the intersecting polygon into the draw control

When you click on the map, it will check if the click point intersects any building polygons from `buildings.geojson` and automatically add the intersecting building to the draw control.

In [ ]:
import json
import geopandas as gpd
from shapely.geometry import Point
from anymap.maplibre import MapLibreMap
import ipywidgets as widgets
from IPython.display import display

## Load Building Data

First, let's load the buildings.geojson file which contains building polygons from the Washington state area.

In [ ]:
# Load buildings data
url = "https://github.com/opengeos/datasets/releases/download/places/las_vegas_buildings.geojson"
buildings_gdf = gpd.read_file(url)

print(f"Loaded {len(buildings_gdf)} building polygons")
print(f"Buildings CRS: {buildings_gdf.crs}")
print(f"Buildings bounds: {buildings_gdf.total_bounds}")

# Display first few buildings
buildings_gdf.head()

In [ ]:
geojson = buildings_gdf.__geo_interface__

In [ ]:
point_gdf = buildings_gdf.copy()
point_gdf["geometry"] = point_gdf.to_crs("EPSG:3857").centroid.to_crs("EPSG:4326")

In [ ]:
point_gdf.head()

In [ ]:
point_geojson = point_gdf.__geo_interface__

In [ ]:
m = MapLibreMap(
    style="liberty", center=[-115.20350534, 36.1209240], zoom=16, add_sidebar=True
)
m.add_basemap("Esri.WorldImagery")
m.add_geojson_layer(
    "building_point",
    point_geojson,
    layer_type="circle",
    paint={"circle-color": "#0000ff", "circle-radius": 5},
)
m.add_draw_control()
m

## Create Interactive Map

Create a map centered in the Washington state area where the buildings are located, and add a draw control.

In [ ]:
# Create output widget to show messages
output = widgets.Output()

In [ ]:
def on_map_clicked(change):
    """Handle map click events."""
    clicked_coords = change["new"]

    with output:
        output.clear_output()

        if not clicked_coords:
            return

        lng = clicked_coords.get("lng")
        lat = clicked_coords.get("lat")

        print(f"Map clicked at: {lng:.6f}, {lat:.6f}")

        # Create point from click coordinates
        click_point = Point(lng, lat)

        # First check if there's already a polygon in draw data that intersects this point
        existing_draw_data = m.get_draw_data()
        existing_features = existing_draw_data.get("features", [])

        # Check if click point intersects any existing drawn features
        for feature in existing_features:
            try:
                from shapely.geometry import shape

                existing_geom = shape(feature["geometry"])
                if existing_geom.intersects(click_point):
                    print(
                        f"⚠️  Click point already intersects an existing drawn feature"
                    )
                    print(
                        f"    Feature: {feature.get('properties', {}).get('name', 'Unnamed')}"
                    )
                    print(f"    Skipping search for new buildings")
                    return
            except Exception as e:
                # Skip if there's an issue with the geometry
                continue

        # Find intersecting building in the buildings dataset
        building = find_intersecting_building(clicked_coords)

        if building is not None:
            print(f"✅ Found intersecting building!")
            print(f"   Building ID: {building['id']}")
            print(f"   Place ID: {building['height']}")

            # Convert building geometry to GeoJSON
            building_geojson = {
                "type": "Feature",
                "geometry": building.geometry.__geo_interface__,
                "properties": {
                    "buildingId": str(building["id"]),
                    "height": str(building["height"]),
                    "clicked_at": f"{lng:.6f}, {lat:.6f}",
                },
            }

            # Add building to draw control
            m.add_draw_data(building_geojson)
            print(f"📍 Added building to draw control")

        else:
            print(f"❌ No building found at this location")


# Observe changes to the clicked attribute
m.observe(on_map_clicked, names="clicked")

display(output)

In [ ]:
def find_intersecting_building(click_coords):
    """Find building polygon that intersects with click point."""
    if not click_coords:
        return None

    lng = click_coords.get("lng")
    lat = click_coords.get("lat")

    if lng is None or lat is None:
        return None

    # Create point from click coordinates
    click_point = Point(lng, lat)

    # Find intersecting buildings
    intersecting = buildings_gdf[buildings_gdf.geometry.intersects(click_point)]

    if len(intersecting) > 0:
        # Return the first intersecting building
        return intersecting.iloc[0]

    return None